<a href="https://colab.research.google.com/github/ruohoruotsi/masakhane/blob/master/en-bin/jw300-baseline/English_to_Edo_Word-level_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "bin" 
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)

In [3]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/en-bin-baseline


In [0]:
# Install opus-tools
! pip install opustools-pkg

     |████████████████████████████████| 81kB 2.5MB/s 


In [0]:
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-iso.xml.gz not found. The following files are available for downloading:

   2 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-iso.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  26 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/iso.zip

 291 MB Total size
./JW300_latest_xml_en-iso.xml.gz ... 100% of 2 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_iso.zip ... 100% of 26 MB


In [0]:
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en 
! mv test.en-$trg.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2020-01-17 04:59:50--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.05s   

2020-01-17 04:59:51 (5.15 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-01-17 04:59:52--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-iso.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Len

In [0]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [0]:
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 5685/243487 lines since contained in test set.


,source_sentence,target_sentence
0,“ Only One of the Many Lives That You Touched ”,“ Omọvo Ahwo Buobu nọ Who Duobọ te Uzuazọ Riẹ ”
1,"BACK in January 1996 , Carol was sick with a b...","EVAỌ January 1996 , Carol ọ jẹ mọ ẹyao ẹvori ."
2,She was in her 60 ’ s and until then had alway...,Ọ kpako te ikpe 60 no yọ oke yena kpobi ọ jọ a...


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [0]:
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144672 sha256=27cc679925e9ea7d499147fe08f2564ad8e4295a570a1a075b46139a669d35bc
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.10 0.00 percent complete
00:00:23.78 0.46 percent complete


00:00:47.40 0.92 percent complete
00:01:10.57 1.39 percent complete


00:01:33.67 1.85 percent complete
00:01:56.94 2.31 percent complete
00:02:20.72 2.77 percent complete
00:02:43.97 3.23 percent complete
00:03:07.38 3.70 percent complete
00:03:31.62 4.16 percent complete
00:03:56.02 4.62 percent complete
00:04:19.06 5.08 percent complete
00:04:42.39 5.54 percent complete
00:05:06.25 6.00 percent complete
00:05:30.46 6.47 percent complete
00:05:53.63 6.93 percent complete
00:06:17.23 7.39 percent complete
00:06:41.67 7.85 percent complete
00:07:05.22 8.31 percent complete
00:07:29.08 8.78 percent complete
00:07:52.78 9.24 percent complete
00:08:16.19 9.70 percent complete
00:08:39.44 10.16 percent complete
00:09:03.31 10.62 percent complete
00:09:26.40 11.09 percent complete
00:09:51.25 11.55 percent complete
00:10:15.14 12.01 percent complete
00:10:38.52 12.47 percent complete
00:11:02.41 12.93 percent complete
00:11:26.41 13.40 percent complete
00:11:50.05 13.86 percent complete
00:12:13.38 14.32 percent complete


00:12:37.46 14.78 percent complete
00:13:01.81 15.24 percent complete
00:13:25.69 15.70 percent complete
00:13:49.63 16.17 percent complete
00:14:13.48 16.63 percent complete
00:14:37.29 17.09 percent complete
00:15:01.50 17.55 percent complete
00:15:25.57 18.01 percent complete
00:15:49.18 18.48 percent complete
00:16:13.51 18.94 percent complete
00:16:37.01 19.40 percent complete
00:17:00.10 19.86 percent complete
00:17:23.41 20.32 percent complete
00:17:47.00 20.79 percent complete
00:18:10.96 21.25 percent complete
00:18:34.67 21.71 percent complete
00:18:59.32 22.17 percent complete
00:19:23.61 22.63 percent complete
00:19:47.81 23.10 percent complete
00:20:12.35 23.56 percent complete
00:20:36.05 24.02 percent complete
00:20:59.50 24.48 percent complete
00:21:23.22 24.94 percent complete
00:21:47.62 25.40 percent complete
00:22:11.51 25.87 percent complete
00:22:35.28 26.33 percent complete
00:22:58.71 26.79 percent complete
00:23:22.48 27.25 percent complete
00:23:46.81 27.71 pe

00:24:10.94 28.18 percent complete
00:24:34.16 28.64 percent complete
00:24:58.34 29.10 percent complete
00:25:22.14 29.56 percent complete
00:25:46.52 30.02 percent complete
00:26:09.90 30.49 percent complete
00:26:34.10 30.95 percent complete
00:26:58.06 31.41 percent complete
00:27:21.26 31.87 percent complete
00:27:45.50 32.33 percent complete
00:28:09.00 32.79 percent complete
00:28:32.87 33.26 percent complete
00:28:57.11 33.72 percent complete
00:29:21.37 34.18 percent complete
00:29:45.41 34.64 percent complete
00:30:08.93 35.10 percent complete
00:30:32.97 35.57 percent complete
00:30:56.46 36.03 percent complete
00:31:19.93 36.49 percent complete
00:31:44.33 36.95 percent complete
00:32:07.88 37.41 percent complete
00:32:32.08 37.88 percent complete
00:32:56.34 38.34 percent complete
00:33:20.79 38.80 percent complete
00:33:44.59 39.26 percent complete
00:34:08.98 39.72 percent complete
00:34:32.93 40.19 percent complete
00:34:56.72 40.65 percent complete
00:35:21.12 41.11 pe

00:40:32.51 47.11 percent complete
00:40:56.64 47.58 percent complete
00:41:20.36 48.04 percent complete
00:41:44.61 48.50 percent complete
00:42:08.32 48.96 percent complete
00:42:31.93 49.42 percent complete
00:42:56.28 49.89 percent complete
00:43:20.24 50.35 percent complete
00:43:44.62 50.81 percent complete
00:44:08.91 51.27 percent complete
00:44:32.54 51.73 percent complete
00:44:56.09 52.19 percent complete
00:45:20.95 52.66 percent complete
00:45:44.71 53.12 percent complete
00:46:08.30 53.58 percent complete
00:46:32.61 54.04 percent complete
00:46:56.86 54.50 percent complete
00:47:20.82 54.97 percent complete
00:47:44.99 55.43 percent complete
00:48:08.38 55.89 percent complete
00:48:32.48 56.35 percent complete
00:48:56.67 56.81 percent complete
00:49:20.35 57.28 percent complete
00:49:44.61 57.74 percent complete
00:50:08.07 58.20 percent complete
00:50:32.40 58.66 percent complete
00:50:56.51 59.12 percent complete
00:51:20.65 59.59 percent complete
00:51:44.56 60.05 pe

01:13:15.72 84.99 percent complete
01:13:39.67 85.45 percent complete
01:14:03.39 85.91 percent complete
01:14:27.32 86.38 percent complete
01:14:51.76 86.84 percent complete
01:15:15.58 87.30 percent complete
01:15:39.33 87.76 percent complete
01:16:03.85 88.22 percent complete
01:16:27.71 88.68 percent complete
01:16:51.58 89.15 percent complete
01:17:15.41 89.61 percent complete
01:17:39.83 90.07 percent complete
01:18:04.13 90.53 percent complete
01:18:28.67 90.99 percent complete
01:18:53.01 91.46 percent complete
01:19:16.73 91.92 percent complete
01:19:41.87 92.38 percent complete
01:20:05.67 92.84 percent complete
01:20:29.44 93.30 percent complete
01:20:53.86 93.77 percent complete
01:21:18.33 94.23 percent complete
01:21:42.36 94.69 percent complete
01:22:06.26 95.15 percent complete
01:22:29.82 95.61 percent complete
01:22:53.83 96.08 percent complete
01:23:17.51 96.54 percent complete
01:23:41.70 97.00 percent complete
01:24:05.69 97.46 percent complete
01:24:30.08 97.92 pe

In [0]:
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [4]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 61, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (39/39), done.
remote: Total 2245 (delta 34), reused 34 (delta 22), pack-reused 2184
Receiving objects: 100% (2245/2245), 2.63 MiB | 16.80 MiB/s, done.
Resolving deltas: 100% (1555/1555), done.
Processing /content/joeynmt
     |████████████████████████████████| 276kB 7.9MB/s 
     |████████████████████████████████| 307kB 14.1MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 215kB 19.8MB/s 
     |████████████████████████████████| 747kB 19.2MB/s 
     |████████████████████████████████| 61kB 9.0MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=72136 sha256=159545bc769a0d181d1eee7bbf6f30321d64b70603b41fe08604e22c8ddc149c
  Stored in directory: /tmp/pip-ephem-wheel-cache-5r8ske78/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel for pyy

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
# # One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# # Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# # Do subword NMT
# from os import path
# os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
# os.environ["tgt"] = target_language

# # Learn BPEs on the training data.
# os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
# ! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# # Apply BPE splits to the development and test data.
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
# ! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# # Create directory, move everyone we care about to the correct location
# ! mkdir -p $data_path
# ! cp train.* $data_path
# ! cp test.* $data_path
# ! cp dev.* $data_path
# ! cp bpe.codes.4000 $data_path
# ! ls $data_path

# # Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
# ! cp train.* "$gdrive_path"
# ! cp test.* "$gdrive_path"
# ! cp dev.* "$gdrive_path"
# ! cp bpe.codes.4000 "$gdrive_path"
# ! ls "$gdrive_path"

# # Create that vocab using build_vocab
# ! sudo chmod 777 joeynmt/scripts/build_vocab.py
# ! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# # Some output
# ! echo "BPE Ẹ̀dó Sentences"
# ! tail -n 5 test.bpe.$tgt
# ! echo "Combined BPE Vocab"
# ! tail -n 10 joeynmt/data/$src$tgt/vocab.txt  # Herman
# ! cp joeynmt/data/$src$tgt/vocab.txt "$gdrive_path"

In [5]:
###### IOHAVOC MODIFICATIONS ==>> CREATE THE VOCAB FOR NON-BPE EXPERIMENTS
from os import path

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py "$gdrive_path/train.$src" "$gdrive_path/train.$tgt" --output_path "$gdrive_path/vocab-nonBPE.txt"

# Some output
! echo "Ẹ̀dó test Sentences"
! tail -n 5 "$gdrive_path/test.$tgt"
! echo "Combined Vocab"
! tail -n 10 "$gdrive_path/vocab-nonBPE.txt"  # iroro

Ẹ̀dó test Sentences
I keghi do bẹghe ẹre wẹẹ , emwa wa danmwehọ ẹsẹsẹmwẹse vbe iran gha tobọ iran bẹghe ẹre wẹẹ , e Baibol ẹre u mobọ loo kevbe wẹẹ , ekhọe hia ẹre u ya hoo ne u ru iyobọ ne iran . ”
Umozo ọghe orhiọn ( Ghee okhuẹn 19 - 20 )
Vbe ẹtin e Jehova , ma gha sẹtin khọnmiotọ yan oghian ima !
( b ) De emwi ne ima khian ziro yan vbe ako iruẹmwi nọ ghi lele ọna ?
( Tie Hibru 11 : 24 - 27 . )
Combined Vocab
payable
Focused
balancing
redeeming
complaint
accord
locate
explanations
counter
Ọgbẹlẹzẹ


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train"
    dev:   "{gdrive_path}/dev"
    test:  "{gdrive_path}/test"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab-nonBPE.txt"
    trg_vocab: "{gdrive_path}/vocab-nonBPE.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    # load_model: "{gdrive_path}/models/{name}_transformer_orig/142000.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 120                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 100          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{gdrive_path}/models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

! cp joeynmt/configs/transformer_$src$tgt.yaml "$gdrive_path"

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [7]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
# !cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml
!python3 -m joeynmt train "$gdrive_path/transformer_$src$tgt.yaml"

2020-01-26 03:56:48,705 Hello! This is Joey-NMT.
2020-01-26 03:56:49,971 Total params: 14526720
2020-01-26 03:56:49,972 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
# !cp -r joeynmt/models/${src}${tgt}_transformer/* "$gdrive_path/models/${src}${tgt}_transformer/"

In [8]:
# Output our validation accuracy epoch 1-30
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

Steps: 100	Loss: 117286.53906	PPL: 152.35403	bleu: 0.00000	LR: 0.00030000	*
Steps: 200	Loss: 111289.65625	PPL: 117.82683	bleu: 0.00000	LR: 0.00030000	*
Steps: 300	Loss: 106089.82031	PPL: 94.29065	bleu: 0.00000	LR: 0.00030000	*
Steps: 400	Loss: 100155.70312	PPL: 73.11843	bleu: 0.00000	LR: 0.00030000	*
Steps: 500	Loss: 95349.11719	PPL: 59.50728	bleu: 0.31248	LR: 0.00030000	*
Steps: 600	Loss: 90747.89062	PPL: 48.85797	bleu: 0.78670	LR: 0.00030000	*
Steps: 700	Loss: 87377.42969	PPL: 42.28699	bleu: 1.72511	LR: 0.00030000	*
Steps: 800	Loss: 84570.64844	PPL: 37.49462	bleu: 1.89084	LR: 0.00030000	*
Steps: 900	Loss: 82261.75781	PPL: 33.96234	bleu: 1.48612	LR: 0.00030000	*
Steps: 1000	Loss: 80139.09375	PPL: 31.00930	bleu: 2.36609	LR: 0.00030000	*
Steps: 1100	Loss: 78887.59375	PPL: 29.39002	bleu: 2.22556	LR: 0.00030000	*
Steps: 1200	Loss: 77106.46875	PPL: 27.23020	bleu: 3.47284	LR: 0.00030000	*
Steps: 1300	Loss: 76134.80469	PPL: 26.11963	bleu: 3.10803	LR: 0.00030000	*
Steps: 1400	Loss: 74919.4218

In [9]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"

2020-01-26 05:36:06,194 Hello! This is Joey-NMT.
2020-01-26 05:36:34,163  dev bleu:   5.99 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-01-26 05:36:49,732 test bleu:   8.24 [Beam search decoding with beam size = 5 and alpha = 1.0]
